# ATR(Average True Range)기반 변동성 돌파 매매 전략

In [1]:
!pip install backtrader
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.3 MB/s eta 0:00:00


In [2]:
import backtrader as bt
import math
import yfinance as yf

In [10]:
# 변동성 돌파 전략 (ATR 기반)
class VolatilityBreakout(bt.Strategy):
    params = dict(
        atr_period=14, # atr 기간
        risk_percent=0.1, # 손절 비율
    )

    def __init__(self):
        # ATR 지표 계산
        self.atr = bt.indicators.ATR(self.data, period=self.p.atr_period)

        # 주문 상태 및 매수/손절 관련 변수 초기화
        self.order = None  # 주문 상태를 저장하는 변수
        self.buy_price = None  # 초기 매수 가격

    def next(self):
        # 주문 상태를 확인
        if self.order:
            return  # 주문이 처리 중이면 대기

        # 변동성 돌파 매매: ATR을 활용한 목표 가격 설정
        target_price = self.data.open[0] + self.atr[0]  # 목표 가격

        if not self.position:  # 포지션이 없는 경우
            # 목표 가격 돌파 시 매수
            if self.data.high[0] >= target_price:
                self.buy_price = target_price
                self.order = self.buy()  # 전액 매수
        else:  # 포지션이 있는 경우
            current_price = self.data.close[0]

            # 손절 조건: 현재 가격이 매수 가격 대비 10% 하락한 경우
            if current_price < self.buy_price * (1 - self.p.risk_percent):
                self.order = self.sell(size=self.position.size)  # 전량 매도

    def log(self, message):
        print(message)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 주문 제출/수락 상태이면 별도 처리 없음
            return

        cur_date = None
        if order.status in [order.Completed]:
            cur_date = order.data.datetime.date(0)
            if order.isbuy():
                self.log(
                    f'{cur_date} [매수 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
                )
            elif order.issell():
                self.log(
                    f'{cur_date} [매도 주문 실행] 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
                )
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(
                f'{cur_date} 주문이 거부되었습니다. 종목: {order.data._name} \t 수량: {order.size} \t 가격: {order.executed.price:.2f}'
            )



In [11]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(VolatilityBreakout)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


2020-01-28 [매수 주문 실행] 종목:  	 수량: 1 	 가격: 8912.52
Final Portfolio Value: 10097090.46
수익률: 100.97%


ATR(Average True Range)기반 변동성 돌파 매매 전략 + 리스크 고정

In [14]:
import backtrader as bt
import yfinance as yf
import datetime

# 리스크 고정 투자법 전략
class VolatilityBreakoutFixedRiskStrategy(bt.Strategy):
    params = (
        ('risk_percent', 0.02),  # 리스크 비율 (2%)
        ('atr_period', 14),  # ATR 계산 기간
        ('atr_multiplier', 2),  # ATR 곱하여 손절 가격 설정
    )

    def __init__(self):
        # ATR 계산 (변동성을 기준으로 손절 가격 설정)
        self.atr = bt.indicators.ATR(self.data, period=self.p.atr_period)

        # 주문 상태 및 매수/손절 관련 변수 초기화
        self.order = None  # 주문 상태 초기화
        self.buy_price = None  # 초기 매수 가격 (초기화 추가)

    def calculate_position_size(self, entry_price, stop_loss_price):
        """포지션 크기 계산: 리스크 고정 투자법 적용"""
        # 계좌에서 허용되는 리스크 금액
        risk_amount = self.broker.getvalue() * self.p.risk_percent
        # 한 주당 손실
        per_unit_risk = abs(entry_price - stop_loss_price)
        # 포지션 크기 계산
        if per_unit_risk > 0:
            position_size = risk_amount / per_unit_risk
            return int(position_size)  # 소수점 제거
        return 0

    def next(self):
        if self.order:  # 주문이 처리 중이면 대기
            return

        if not self.position:  # 포지션이 없는 경우
            entry_price = self.data.close[0]
            stop_loss_price = entry_price - (self.atr[0] * self.p.atr_multiplier)

            # 목표 가격 돌파 시 매수
            if self.data.close[0] > self.data.open[0]:  # 간단한 상승 신호
                position_size = self.calculate_position_size(entry_price, stop_loss_price)
                if position_size > 0:
                    self.buy_price = entry_price  # 매수 가격 기록
                    self.order = self.buy(size=position_size)
        else:  # 포지션이 있는 경우
            current_price = self.data.close[0]

            # 손절 조건: 현재 가격이 손절 가격보다 낮아지면 매도
            if current_price < self.buy_price - (self.atr[0] * self.p.atr_multiplier):
                self.order = self.sell(size=self.position.size)

    def notify_order(self, order):
        if order.status in [order.Completed, order.Canceled, order.Rejected]:
            self.order = None  # 주문 완료 후 초기화




In [15]:
if __name__ == '__main__':

  cerebro = bt.Cerebro()
  cerebro.addstrategy(VolatilityBreakoutFixedRiskStrategy)
  cerebro.broker.setcommission(commission=0.003) # 수수료 0.3%
  cerebro.broker.setcash(10_000_000)

  print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
  data = yf.download('BTC-USD', start='2020-01-01', end='2024-12-17')
  # 열 이름 전처리
  data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
  data_bt = bt.feeds.PandasData(dataname=data)
  cerebro.adddata(data_bt)
  cerebro.run()
  print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
  print(f'수익률: {cerebro.broker.getvalue() / 10_000_000 * 100:.2f}%')

[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value: 10000000.00


최종 포트폴리오 가치: 23244720.01
Final Portfolio Value: 23244720.01
수익률: 232.45%
